In [1]:
import os
import cmdstanpy
from cmdstanpy import Model, StanFit, cmdstan_path
from cmdstanpy import Model, StanFit
import ujson as json

<h3>Example:</h3>

Running the model on Men's 2014 World Cup

In [3]:
with open('worldcup_2014.data.json', 'r') as fd:
    data_dict = json.load(fd)
data_dict.keys()

# stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
# stan_wc.compile()
# stan_wc

# worldcup_fit = stan_wc.sample(data=data_dict, chains=4)
# worldcup_fit.diagnose()
# worldcup_fit.summary()

dict_keys(['I', 'N', 'team_1', 'team_2', 'score_1', 'score_2', 'prior_score'])

In [5]:
with open('countries.data.json', 'r') as fd_country:
    country_dict = json.load(fd_country)
country_dict.keys()

dict_keys(['countries'])

<h3> Attempting the same for the Women' World Cup Data </h3>

(Import the data... and massaging it into the format stan needs it in.) 

Data input needs to be in a dictionary format;
Keys in the dictionary are the names of the variables in the data block of the stan model;

In [6]:
import pandas as pd
matches = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_matches.csv")
forecasts = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_forecasts.csv")
print(matches.columns, forecasts.columns)

Index(['date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2',
       'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1',
       'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object') Index(['forecast_timestamp', 'team', 'group', 'spi', 'global_o', 'global_d',
       'sim_wins', 'sim_ties', 'sim_losses', 'sim_goal_diff', 'goals_scored',
       'goals_against', 'group_1', 'group_2', 'group_3', 'group_4',
       'make_round_of_16', 'make_quarters', 'make_semis', 'make_final',
       'win_league', 'timestamp'],
      dtype='object')


In [7]:
matches.iloc[44:]['score1']

44   NaN
45   NaN
46   NaN
47   NaN
Name: score1, dtype: float64

In [8]:
#quarter-final info not yet filled in! Wikipedia to the rescue:[0,1,0,1]
matches.score1.iloc[[44, 45, 46, 47]] = [0,1,0,1]
matches.score2.iloc[[44, 45, 46, 47]] = [3,2,2,2]

/Users/nitya/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
matches.iloc[44:]

,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
44,2019-06-27,352,FIFA Women's World Cup Final,Norway Women,England Women,84.10,92.65,0.3095,0.6905,0.0,0.81,1.47,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
45,2019-06-28,352,FIFA Women's World Cup Final,France Women,USA Women,96.79,98.50,0.4567,0.5433,0.0,1.25,1.41,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
46,2019-06-29,352,FIFA Women's World Cup Final,Italy Women,Netherlands Women,77.41,93.32,0.2146,0.7854,0.0,0.90,2.10,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
47,2019-06-29,352,FIFA Women's World Cup Final,Germany Women,Sweden Women,94.89,88.46,0.6271,0.3729,0.0,1.26,0.85,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
len(matches.team2.unique()), len(matches.team1.unique()), len(forecasts.team.unique())

(24, 24, 24)

In [11]:
#Getting the scores before the tournament
prior_tournament = forecasts[forecasts['forecast_timestamp']=='2019-05-16 18:22:11 UTC']
countries = prior_tournament[['team', 'spi']].sort_values(by=['spi'], ascending=False)
wwc2019_country = countries['team'].values
print(wwc2019_country)

['USA' 'France' 'Germany' 'Australia' 'Netherlands' 'England' 'Japan'
 'Canada' 'Brazil' 'Sweden' 'Spain' 'New Zealand' 'Norway' 'China'
 'South Korea' 'Italy' 'Nigeria' 'Cameroon' 'South Africa' 'Jamaica'
 'Scotland' 'Thailand' 'Chile' 'Argentina']


In [12]:
spi = dict(zip(wwc2019_country, list(range(1, len(wwc2019_country)+1))))

In [13]:
spi

{'USA': 1,
 'France': 2,
 'Germany': 3,
 'Australia': 4,
 'Netherlands': 5,
 'England': 6,
 'Japan': 7,
 'Canada': 8,
 'Brazil': 9,
 'Sweden': 10,
 'Spain': 11,
 'New Zealand': 12,
 'Norway': 13,
 'China': 14,
 'South Korea': 15,
 'Italy': 16,
 'Nigeria': 17,
 'Cameroon': 18,
 'South Africa': 19,
 'Jamaica': 20,
 'Scotland': 21,
 'Thailand': 22,
 'Chile': 23,
 'Argentina': 24}

In [14]:
import numpy as np
mean = np.mean(countries['spi'])
std = np.std(countries['spi'])
countries['prior_score'] = [(x - mean)/std for x in countries['spi']]

In [15]:
matches['team1'] = matches['team1'].apply(lambda x: x.split('Women')[0].rstrip())
matches['team2'] = matches['team2'].apply(lambda x: x.split('Women')[0].rstrip())

In [16]:
#matches = matches.replace('New', 'New Zealand')

In [17]:
matches['team1']

0           France
1          Germany
2            Spain
3           Norway
4        Australia
5           Brazil
6          England
7        Argentina
8           Canada
9      New Zealand
10           Chile
11             USA
12         Nigeria
13         Germany
14          France
15       Australia
16    South Africa
17           Japan
18         Jamaica
19         England
20     Netherlands
21          Canada
22          Sweden
23             USA
24           China
25    South Africa
26         Nigeria
27     South Korea
28           Italy
29         Jamaica
30        Scotland
31           Japan
32        Cameroon
33     Netherlands
34          Sweden
35        Thailand
36         Germany
37          Norway
38         England
39          France
40           Spain
41          Sweden
42           Italy
43     Netherlands
44          Norway
45          France
46           Italy
47         Germany
Name: team1, dtype: object

In [18]:
N = len(matches)
I = len(wwc2019_country)

In [19]:
mydict = dict({'I': I,
               'N': N,
                'team_1': matches['team1'].values,
               'team_2': matches['team2'].values,
               'score_1': matches['score1'].values,
               'score_2': matches['score2'].values,
              'prior_score': countries['prior_score'].values})

In [20]:
mydict['team_2']

array(['South Korea', 'China', 'South Africa', 'Nigeria', 'Italy',
       'Jamaica', 'Scotland', 'Japan', 'Cameroon', 'Netherlands',
       'Sweden', 'Thailand', 'South Korea', 'Spain', 'Norway', 'Brazil',
       'China', 'Scotland', 'Italy', 'Argentina', 'Cameroon',
       'New Zealand', 'Thailand', 'Chile', 'Spain', 'Germany', 'France',
       'Norway', 'Brazil', 'Australia', 'Argentina', 'England',
       'New Zealand', 'Canada', 'USA', 'Chile', 'Nigeria', 'Australia',
       'Cameroon', 'Brazil', 'USA', 'Canada', 'China', 'Japan', 'England',
       'USA', 'Netherlands', 'Sweden'], dtype=object)

In [21]:
mydict['team_1'] = [spi.get(x) for x in mydict['team_1']]
mydict['team_2'] = [spi.get(x) for x in mydict['team_2']]

In [23]:
stan_wc = Model(stan_file='worldcup_rep_ag.stan')
stan_wc.compile()
stan_wc

INFO:cmdstanpy:stan to c++ (worldcup_rep_ag.hpp)
INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/nitya/Work/test_tutorial/worldcup_rep_ag


Model(name=worldcup_rep_ag,  stan_file="worldcup_rep_ag.stan", exe_file="/Users/nitya/Work/test_tutorial/worldcup_rep_ag")

In [24]:
worldcup_fit = stan_wc.sample(data=mydict, chains=4)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


In [25]:
worldcup_fit.diagnose()


Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


'Processing csv files: /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpbnmgui9a/stan-worldcup_rep_ag-draws-1-x768u12f.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpbnmgui9a/stan-worldcup_rep_ag-draws-2-1qatj7cr.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpbnmgui9a/stan-worldcup_rep_ag-draws-3-u91_nk5l.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpbnmgui9a/stan-worldcup_rep_ag-draws-4-18z2n0ua.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.'

In [26]:
worldcup_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-52.576400,0.280390,6.890810,-64.935100,-52.088700,-42.042500,603.970,499.134,1.006940
beta,0.942710,0.005871,0.266341,0.502589,0.943266,1.372870,2058.030,1700.800,0.999849
alpha[1],0.978142,0.011738,0.688052,-0.187337,1.006160,2.057160,3435.840,2839.450,1.001160
alpha[2],0.327845,0.009667,0.646196,-0.713839,0.319274,1.394660,4468.120,3692.550,1.000450
alpha[3],0.407858,0.009957,0.659097,-0.663937,0.418688,1.525000,4381.270,3620.770,1.000100
alpha[4],-0.169733,0.010956,0.684803,-1.288220,-0.179461,0.980128,3906.710,3228.590,0.999660
alpha[5],0.453820,0.009846,0.622770,-0.553438,0.454753,1.458990,4000.330,3305.950,1.000320
alpha[6],0.549165,0.011909,0.689844,-0.609845,0.566894,1.644890,3355.620,2773.160,1.000450
alpha[7],-0.763160,0.012262,0.741430,-2.045420,-0.724649,0.377331,3655.920,3021.330,0.999660
